* Scraping postal code in wiki *

In [96]:
!pip install geocoder
!pip install cssselect
import pandas as pd
import requests
import lxml.html as lh

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbor'])
i=0
#For each row, store each first element (header) and an empty list
j = 0
for a in range(0,20) :
  for t in tr_elements[a]:
      i+=1
      name=t.text_content().replace('\n','').replace('/',',')      
      if 'Not' not in name[3:16] :
      
        if name.find('(') < 0 :
          df.loc[j] = [name[0:3], name[3:], name[3:]]
        else:
          df.loc[j] = [name[0:3], name[3:name.find('(')], name[name.find('(')+1:name.find(')')]]        
        j = j + 1

# get geospatial data
df2 = pd.read_csv("http://cocl.us/Geospatial_data")

# Join two data frame
df_final = df.set_index('Postal Code').join(df2.set_index('Postal Code'))


df_final.head()



,Borough,Neighbor,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [97]:
df_final.shape

(103, 4)